# Single phase Debora: public (coolprop) equations of state

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import os
from IPython.display import display
from string import Template
import pandas as pd
import matplotlib.cm as cm
import matplotlib as mpl

from trustutils import run 
from trustutils import visit
from trustutils.jupyter import plot

run.TRUST_parameters()

In [ ]:
force_recalculation = True 

number_of_partitions = 2

carac_essai = pd.read_csv(f"src/Kledy_Monophasique.csv", sep=";", skiprows=[1])

import CoolProp.CoolProp as CP
carac_essai["u0"]       = [ row["G"]/CP.PropsSI("DMASS", "P", row["P"]*1.e5, "T", row["Te"]+273.15, "R134a") for n, row in carac_essai.iterrows()]
carac_essai["Tsat"]     = [ CP.PropsSI("T", "P", row["P"]*1.e5,  "Q", 0, "R134a") for n, row in carac_essai.iterrows()]
carac_essai["CL_k"]     = 0.01*np.array(carac_essai["u0"])**2
carac_essai["CL_om"]    = 0.5 *np.array(carac_essai["u0"])/0.0192
carac_essai["flux"]     =  np.array(carac_essai["qp"])*1.e3
carac_essai["PPa"]     =  np.array(carac_essai["P"])*1.e5

nb_pas_dt_max = 1

if force_recalculation or not os.path.exists('build'):
    run.reset()
    origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
    for n, row in carac_essai.iterrows() :
        dict_substitute = {"u_0"  : f"{row['u0']}*(t_MIN_1)" ,
                            "k0" : row["CL_k"] ,
                            "om0": row["CL_om"] ,
                            "P_surf" : f"{row['flux']}*(t_MIN_1)",
                            "Tl0" : row["Te"],
                            "Tg0" : row["Tsat"],
                            "P0" : row["PPa"],
                            "rho" : CP.PropsSI("D", "P", row["PPa"], "T", row["Te"] + 273.15, "R134a"),
                            "mu" : CP.PropsSI("V", "P", row["PPa"], "T", row["Te"] + 273.15, "R134a"),
                            "cp" : CP.PropsSI("C", "P", row["PPa"], "T", row["Te"] + 273.15, "R134a"),
                            "lambda" : CP.PropsSI("L", "P", row["PPa"], "T", row["Te"] + 273.15, "R134a"),
                            "beta" : CP.PropsSI("d(D)/d(T)|P", "P", row["PPa"], "T", row["Te"] + 273.15, "R134a") / CP.PropsSI("D", "P", row["PPa"], "T", row["Te"] + 273.15, "R134a"),
                            "nb_pas_dt_max" : nb_pas_dt_max
                        }
        name_run = f"{row['Essai']}".replace('&', '_')
        print(name_run)
        
        tc = run.addCaseFromTemplate(templateData=f"jdd.data",targetDirectory=f"{name_run}",dic=dict_substitute,nbProcs=number_of_partitions,targetData=f"{name_run}.data")
                    
        if number_of_partitions > 1:
            tc.partition()

    run.printCases()
    run.runCases()
else:
    print('No recalculation, reusing old results!')

In [ ]:
def loadText(data, index_column=0, nb_column=-1, transpose=True, dtype="float", skiprows=0):   
    if nb_column == -1:
        nb = None
    else:
        nb = index_column + nb_column
    try:
        if transpose:
            matrix = np.loadtxt(data, dtype=dtype, skiprows=skiprows).T[index_column:nb]
        else:
            matrix = np.loadtxt(data, dtype=dtype, skiprows=skiprows)[index_column:nb]
    except:
        matrix = np.loadtxt(data, dtype=dtype, skiprows=skiprows)                
    return matrix


In [ ]:
plt.rcParams.update({
    'text.usetex': False,
    'font.family': 'serif',
})

t_mesure=-1
nPoints=20
absc = np.linspace(1/(2*nPoints), 1-1/(2*nPoints), 20)

par = ""
if number_of_partitions > 1 : par = "PAR_"

fig = plt.figure(figsize = (3*7, 3*2))
axs = fig.subplots(2,7)

markers = { "mono" : "--", "mono_solide" : "-", "di" : ":" }

for n, row in carac_essai.iterrows() :
    axs[0,n].set_title(row['Essai'])
    # axs[0,n].set_xlabel(r"$r_+$")
    # axs[0,n].set_ylabel(r"$u_z$")
    # axs[1,n].set_xlabel(r"$r_+$")
    # axs[1,n].set_ylabel(r"T")
    axs[0,n].set_xlim(-.05, 1.1)
    axs[1,n].set_xlim(-.05, 1.1)
    
    name_run = f"{row['Essai']}".replace('&', '_')
    vzSim    = np.array(loadText(f"{run.BUILD_DIRECTORY}/{name_run}/{par}{name_run}_VITESSE.son"))[3::3,t_mesure]
    TSim     = np.array(loadText(f"{run.BUILD_DIRECTORY}/{name_run}/{par}{name_run}_TEMPERATURE.son"))[1::,t_mesure]
    axs[0,n].plot(absc, vzSim, "b--")
    axs[1,n].plot(absc, TSim, "b--")
    
    if os.path.exists(f"{run.BUILD_DIRECTORY}/{name_run}/{par}{name_run}_TEMPERATURE_SOL_SORTIE.son") :
        TsSim     = np.array(loadText(f"{run.BUILD_DIRECTORY}/{name_run}/{par}{name_run}_TEMPERATURE_SOL_SORTIE.son"))[1::,t_mesure]
        axs[1,n].plot(np.linspace(1+1/9.6*1/20, 1+1/9.6*19/20, 10), TsSim  , "b")

    manip = pd.read_csv(f"{run.BUILD_DIRECTORY}/data/{row['Essai']}.csv", sep=";")
    axs[0,n].plot(manip["ru"], manip["u"], "ob")
    axs[1,n].plot(manip["rt"][:-1], manip["T"][:-1], "ob")
    axs[1,n].plot(manip["rt"][-1:], manip["T"][-1:], "sb")
    axs[0,n].set_ylim(float(row['Essai'][1])*.5, float(row['Essai'][1])*1.1)
    axs[1,n].set_ylim(int(min(manip["T"][:-1])*.9), int(max(manip["T"][-1:])*1.1))
    # axs[0,n].legend()
    # axs[1,n].legend()

# fig.tight_layout()
fig.savefig(f"{run.BUILD_DIRECTORY}/Single_phase_Debora_R134a.pdf")


In [ ]:
t_mesure=-1
nPoints=20
absc = np.linspace(1/(2*nPoints), 1-1/(2*nPoints), 20)

par = ""
if number_of_partitions > 1 : par = "PAR_"

fig = plt.figure(figsize = (4*2, 4))
axs = fig.subplots(1,2)

norm_cm = mpl.colors.Normalize(-1, 81)
cmap = cm.jet
m = cm.ScalarMappable(norm=norm_cm, cmap=cmap)

for n, row in carac_essai.iterrows() :
    if (row['Essai'][1]=='3') :
        color_loc = m.to_rgba(float(row['Essai'][3:]))
        name_run = f"{row['Essai']}".replace('&', '_')
        vzSim    = np.array(loadText(f"build/{name_run}/{par}{name_run}_VITESSE.son"))[3::3,t_mesure]
        TSim     = np.array(loadText(f"build/{name_run}/{par}{name_run}_TEMPERATURE.son"))[1::,t_mesure]
        axs[0].plot(absc, TSim-TSim[0]  , color=color_loc)
        axs[1].plot(absc, vzSim/vzSim[0], color=color_loc)

        if os.path.exists(f"build/{name_run}/{par}{name_run}_TEMPERATURE_SOL_SORTIE.son") :
            TsSim     = np.array(loadText(f"build/{name_run}/{par}{name_run}_TEMPERATURE_SOL_SORTIE.son"))[1::,t_mesure]
            axs[0].plot(np.linspace(1+1/9.6*1/20, 1+1/9.6*19/20, 10), TsSim-TSim[0]  , markers[nfluid], color=color_loc, linewidth = linewid[nfluid])
            
        manip = pd.read_csv(f"src/data/{row['Essai']}.csv", sep=";")
        axs[0].plot(manip["rt"][:-1], manip["T"][:-1]-manip["T"][0], "o", color=color_loc, label = f"{row['Essai']} \n " + r"$T_c=$" + f"{manip['T'][0]}"[:5])
        axs[0].plot(manip["rt"][-1:], manip["T"][-1:]-manip["T"][0], "s", color=color_loc)
        axs[1].plot(manip["ru"], manip["u"]/manip["u"][0], "o", color=color_loc, label = f"{row['Essai']}")

axs[0].plot([-2,-3], [-2,-3], color="k")
axs[1].plot([-2,-3], [-2,-3], color="k")

axs[0].set_ylim(-1, 22)
axs[0].set_xlabel(r"$r_+$")
axs[0].set_ylabel(r"$T-T_C$")
axs[0].set_xlim(-.05, 1.05)
axs[0].legend()

axs[1].set_ylim(.5, 1.1)
axs[1].set_ylabel(r"$u_z$")
axs[1].set_xlabel(r"$r_+$")
axs[1].set_xlim(-.05, 1.05)
axs[1].legend()


fig.tight_layout()
fig.savefig("build/Single_phase_Debora_R134a_changing_Q.pdf")


In [ ]:
t_mesure=-1
nPoints=20
absc = np.linspace(1/(2*nPoints), 1-1/(2*nPoints), 20)

par = ""
if number_of_partitions > 1 : par = "PAR_"

fig = plt.figure(figsize = (4*2, 4))
axs = fig.subplots(1,2)

norm_cm = mpl.colors.Normalize(.5, 3.5)
cmap = cm.jet
m = cm.ScalarMappable(norm=norm_cm, cmap=cmap)

for n, row in carac_essai.iterrows() :
    if (row['Essai'][3:]=='20') :
        color_loc = m.to_rgba(float(row['Essai'][1]))
        name_run = f"{row['Essai']}".replace('&', '_')
        vzSim    = np.array(loadText(f"build/{name_run}/{par}{name_run}_VITESSE.son"))[3::3,t_mesure]
        TSim     = np.array(loadText(f"build/{name_run}/{par}{name_run}_TEMPERATURE.son"))[1::,t_mesure]
        axs[0].plot(absc, TSim-TSim[0]  , color=color_loc)
        axs[1].plot(absc, vzSim/vzSim[0], color=color_loc)
    
        if os.path.exists(f"build/{name_run}/{par}{name_run}_TEMPERATURE_SOL_SORTIE.son") :
            TsSim     = np.array(loadText(f"build/{name_run}/{par}{name_run}_TEMPERATURE_SOL_SORTIE.son"))[1::,t_mesure]
            axs[0].plot(np.linspace(1+1/9.6*1/20, 1+1/9.6*19/20, 10), TsSim-TSim[0], color=color_loc)
            
        manip = pd.read_csv(f"src/data/{row['Essai']}.csv", sep=";")
        axs[0].plot(manip["rt"][:-1], manip["T"][:-1]-manip["T"][0], "o", color=color_loc, label = f"{row['Essai']} \n " + r"$T_c=$" + f"{manip['T'][0]}"[:5])
        axs[0].plot(manip["rt"][-1:], manip["T"][-1:]-manip["T"][0], "s", color=color_loc)
        axs[1].plot(manip["ru"], manip["u"]/manip["u"][0], "o", color=color_loc, label = f"{row['Essai']}")

axs[0].plot([-2,-3], [-2,-3], color="k")
axs[1].plot([-2,-3], [-2,-3], color="k")


axs[0].set_ylim(-1, 13)
axs[0].set_xlabel(r"$r_+$")
axs[0].set_ylabel(r"$T-T_C$")
axs[0].set_xlim(-.05, 1.05)
axs[0].legend()

axs[1].set_ylim(.5, 1.1)
axs[1].set_ylabel(r"$u_z$")
axs[1].set_xlabel(r"$r_+$")
axs[1].set_xlim(-.05, 1.05)
axs[1].legend()


fig.tight_layout()
fig.savefig("build/Single_phase_Debora_R134a_changing_G.pdf")
